## Scraping MVP voting information and downloading data

In [ ]:
import cloudscraper
import os
import time

years = list(range(1991, 2025))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

scraper = cloudscraper.create_scraper()

os.makedirs("mvp", exist_ok=True)

for year in years:
    url = url_start.format(year)
    data = scraper.get(url)
    
    with open(f"mvp/{year}.html", "w", encoding="utf-8") as f:
        f.write(data.text)

    time.sleep(10)

In [27]:
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup

dfs = []

for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(StringIO(str(mvp_table)))[0]
    mvp_df["Year"] = year
    
    dfs.append(mvp_df)

In [30]:
mvps = pd.concat(dfs)

mvps.to_csv("mvps.csv")

## Scraping player stats and downloading data

In [35]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

scraper = cloudscraper.create_scraper()

os.makedirs("mvp", exist_ok=True)

for year in years:
    url = player_stats_url.format(year)
    data = scraper.get(url)
    
    with open(f"player/{year}.html", "w", encoding="utf-8") as f:
        f.write(data.text)

    time.sleep(5)

In [37]:
dfs = []

for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    #soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(StringIO(str(player_table)))[0]
    player_df["Year"] = year
    
    dfs.append(player_df)

In [41]:
players = pd.concat(dfs)

players.to_csv("players.csv")